# Исследование уровня лояльности клиентов

В рамках данного исследования проведен анализ информации о текущем уровне NPS среди клиентов телекоммуникационной компании, которая оказывает услуги на территории СНГ. 

**Цель** Провести анализ результатов опроса с оценкой текущего уровня NPS среди клиентов телекоммуникационной компании и показать, как этот уровень меняется в зависимости от пользовательских признаков. Результаты исследования оформить в виде презентации.

**Описание данных** Исследование проведено на основе данных, предоставленных компанией. Для определения уровня лояльности клиентам задавали вопрос: "Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым".
Оценки были разделены на три группы:
9-10 баллов — "cторонники" (англ. promoters);
7-8 баллов — "нейтралы" (англ. passives);
0-6 баллов — "критики" (англ. detractors).

Помимо уровня лояльности в данных отражен ряд признаков пользователей, в том числе:

- пол и возраст,
- страна и город проживания,
- тип используемого устройства и операцинной системы, 
- объем трафика,
- количество дней «жизни» клиента,
- статус клиента (новый/старый).

Некоторые признаки отражены в виде сегментов (напр., трафик), другие имеют конкретные значения (напр. тип устройства).

Данные представляют собой базу данных из нескольких таблиц с результатами опроса клиентов. Данные находятся в СУБД SQLite, где база данных представлена файлом.  В базе имеются таблицы user, location, age_segment, traffic_segment, lifetime_segment, которые связаны друг с другом с помощью id разных признаков.


**Содержание исследования**

На первом этапе исследования получен доступ к базе данных. Далее данные преобразованы с помощью SQL-запросов, а вся необходимая информация из разных таблиц была собрана в одну витрину. Данные были сохранены в датафрейм в pandas. Затем был проведен анализ признаков и поиск взавимосвязей. Результаты оформлены в виде презентации с использованием Tableau. 

В рамках исследования были сделаны выводы по следующим вопросам:

- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

## Подключение к базе. Выгрузка данных

Для подключения к базе данных достаточно иметь доступ к файлу с расширением .db. 
Также необходимо собрать в одну витрину данные из разных таблиц.  Эту витрину мы будем использовать для построения дашборда и офромления результатов.

In [11]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [12]:
# подключаемся к базе данных

path_to_db_local = '-------'
path_to_db_platform = '-------'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [13]:
# готовим SQL-запрос для выгрузки необходимых полей

query = query = """
SELECT us.user_id,
       us.lt_day,
       CASE WHEN lt_day < 365 THEN 'new_client' 
            WHEN lt_day >= 365 THEN 'old_client'
            END AS is_new, 
       us.age,
       CASE WHEN us.gender_segment = 1 THEN 'female' 
            WHEN us.gender_segment = 0 THEN 'male'
            END AS gender_segment,
       LOWER (us.os_name) AS os_name,
       LOWER (us.cpe_type_name) AS cpe_type_name,
       loc.country AS country,
       loc.city AS city,
       SUBSTR(age.title, 4) AS age_segment,
       SUBSTR(traffic.title, 4) AS traffic_segment,
       SUBSTR(lifetime.title, 4) AS lifetime_segment,
       us.nps_score,     
       CASE WHEN 0 < us.nps_score AND us.nps_score <= 6 THEN 'detractor' 
            WHEN 7 <= us.nps_score AND us.nps_score <= 8 THEN 'passive'
            WHEN 9 <= us.nps_score AND us.nps_score <= 10 THEN 'promoter'
            ELSE 'unknown'
            END AS nps_group
FROM user AS us
LEFT OUTER JOIN location AS loc ON us.location_id = loc.location_id
LEFT OUTER JOIN age_segment AS age ON us.age_gr_id = age.age_gr_id
LEFT OUTER JOIN traffic_segment AS traffic ON us.tr_gr_id = traffic.tr_gr_id
LEFT OUTER JOIN lifetime_segment AS lifetime ON us.lt_gr_id = lifetime.lt_gr_id;

"""

In [14]:
# создаём датафрейм по данным запроса
df = pd.read_sql(query, engine)

## Обзор данных

In [10]:
# выводим первые строки и общую информацию о таблице 
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old_client,45.0,female,android,smartphone,Россия,Уфа,45-54,1-5,36+,10,promoter
1,A001WF,2344,old_client,53.0,male,android,smartphone,Россия,Киров,45-54,1-5,36+,10,promoter
2,A003Q7,467,old_client,57.0,male,android,smartphone,Россия,Москва,55-64,20-25,13-24,10,promoter


С помощью SQL-запроса мы выгрузили из таблиц базы данных все необходимые поля для анализа. Далее получившийся датафрейм сохраняем в виде файла формата *csv*. На основе получившегося датафрейма будем строить дашборд в Tableau.

Общая информация о датасете:

| **Поле** | **Описание** |
|:--------|:--------|
|user_id |  Идентификатор клиента | 
|lt_day |  	Количество дней «жизни» клиента |  
|is_new |  Поле хранит информацию о том, является ли клиент новым |  
|age |  Возраст  |  
|gender_segment |  Пол (значения преобразованы в текстовый вид) |  
|os_name |  Тип операционной системы | 
|cpe_type_name |  Тип устройства | 
|country |  Страна проживания |  
|city |  Город проживания | 
|age_segment |  Возрастной сегмент  |  
|traffic_segment |  Сегмент по объему потребляемого трафика  | 
|lifetime_segment |  Сегмент по количеству дней «жизни» (Клиента считают новым, если количество дней «жизни» составляет не более 365 дней) |  
|nps_score |  Оценка клиента в NPS-опросе  | 
|nps_group |  Данные о том, к какой группе относится оценка клиента в опросе  |  

In [ ]:
# сохраненяем таблицу в формате csv
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Презентация

Ссылка на презентацию с результатами: https://public.tableau.com/views/NPSproject_17151924148380/NPS?:language=en-US&:sid=&:display_count=n&:origin=viz_share_link